In [ ]:
# EXEMPLO PRÁTICO PARA GOOGLE COLAB
# Execute este código célula por célula

# 1. INSTALAR DEPENDÊNCIAS
!pip install pandas colorama difflib

# 2. IMPORTAR E CONFIGURAR
import pandas as pd
import io
from IPython.display import Markdown, display
import re
from typing import List, Dict, Tuple
from difflib import SequenceMatcher
import difflib

class ContentValidator:
    def __init__(self):
        self.original_content = {}
        self.generated_content = ""
        self.missing_content = []
        self.added_content = []
        self.coverage_report = {}

    def load_original_data(self, csv_path: str):
        """Carrega os dados originais do CSV"""
        df = pd.read_csv(csv_path)

        # Filtra apenas as linhas com respostas válidas
        valid_rows = df[
            (df['answer'].notna()) &
            (df['answer'] != '')
        ]

        for idx, row in valid_rows.iterrows():
            key = f"item_{idx}"
            self.original_content[key] = {
                'answer': row['answer']
            }

    def load_generated_content(self, generated_text: str):
        """Carrega o conteúdo gerado"""
        self.generated_content = generated_text

    def normalize_text(self, text: str) -> str:
        """Normaliza texto para comparação mais flexível"""
        # Remove acentos básicos
        replacements = {
            'á': 'a', 'à': 'a', 'ã': 'a', 'â': 'a', 'ä': 'a',
            'é': 'e', 'è': 'e', 'ê': 'e', 'ë': 'e',
            'í': 'i', 'ì': 'i', 'î': 'i', 'ï': 'i',
            'ó': 'o', 'ò': 'o', 'õ': 'o', 'ô': 'o', 'ö': 'o',
            'ú': 'u', 'ù': 'u', 'û': 'u', 'ü': 'u',
            'ç': 'c', 'ñ': 'n'
        }

        text = text.lower()
        for old, new in replacements.items():
            text = text.replace(old, new)

        # Remove pontuação e normaliza espaços
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    def extract_key_concepts(self, text: str) -> List[str]:
        """Extrai conceitos-chave do texto (palavras e frases importantes)"""
        normalized = self.normalize_text(text)

        # Palavras importantes (substantivos, verbos, adjetivos)
        words = normalized.split()

        # Filtrar palavras muito comuns
        stopwords = {
            'a', 'e', 'o', 'de', 'da', 'do', 'em', 'para', 'com', 'por', 'que', 'se', 'na', 'no',
            'um', 'uma', 'os', 'as', 'dos', 'das', 'nos', 'nas', 'ao', 'aos', 'pela', 'pelo',
            'mas', 'ou', 'quando', 'onde', 'como', 'muito', 'mais', 'menos', 'ate', 'seu', 'sua',
            'seus', 'suas', 'meu', 'minha', 'meus', 'minhas', 'era', 'foi', 'ser', 'ter', 'estar',
            'foi', 'tinha', 'tem', 'tinha', 'eram', 'sendo', 'sido', 'ja', 'sempre', 'nunca',
            'tambem', 'ainda', 'depois', 'antes', 'entao', 'isso', 'essa', 'este', 'esta',
            'aquele', 'aquela', 'dele', 'dela', 'deles', 'delas', 'me', 'te', 'lhe', 'nos', 'vos'
        }

        # Palavras importantes (pelo menos 3 caracteres e não stopwords)
        key_words = [word for word in words if len(word) >= 3 and word not in stopwords]

        # Frases importantes (sequências de 2-4 palavras)
        key_phrases = []
        for i in range(len(key_words) - 1):
            phrase = ' '.join(key_words[i:i+2])
            if len(phrase) > 8:  # Frases com mais de 8 caracteres
                key_phrases.append(phrase)

        # Combinar palavras e frases
        return key_words + key_phrases

    def calculate_similarity(self, text1: str, text2: str) -> float:
        """Calcula similaridade entre dois textos usando diferentes métricas"""
        norm1 = self.normalize_text(text1)
        norm2 = self.normalize_text(text2)

        # Similaridade básica usando SequenceMatcher
        basic_similarity = SequenceMatcher(None, norm1, norm2).ratio()

        # Similaridade baseada em conceitos-chave
        concepts1 = set(self.extract_key_concepts(text1))
        concepts2 = set(self.extract_key_concepts(text2))

        if not concepts1 or not concepts2:
            concept_similarity = 0.0
        else:
            intersection = len(concepts1 & concepts2)
            union = len(concepts1 | concepts2)
            concept_similarity = intersection / union if union > 0 else 0.0

        # Peso maior para similaridade de conceitos
        final_similarity = (basic_similarity * 0.3) + (concept_similarity * 0.7)

        return final_similarity

    def check_content_presence(self, original_answer: str, generated_text: str) -> Dict:
        """Verifica se o conteúdo original está presente no texto gerado"""
        # Extrai conceitos-chave do texto original
        original_concepts = self.extract_key_concepts(original_answer)

        # Normaliza o texto gerado
        normalized_generated = self.normalize_text(generated_text)

        # Verifica quantos conceitos estão presentes
        present_concepts = []
        missing_concepts = []

        for concept in original_concepts:
            if concept in normalized_generated:
                present_concepts.append(concept)
            else:
                # Verifica similaridade parcial
                found_similar = False
                for word in normalized_generated.split():
                    if self.calculate_similarity(concept, word) > 0.8:
                        present_concepts.append(concept)
                        found_similar = True
                        break

                if not found_similar:
                    missing_concepts.append(concept)

        total_concepts = len(original_concepts)
        present_count = len(present_concepts)

        coverage_percent = (present_count / total_concepts) * 100 if total_concepts > 0 else 0

        return {
            'total_concepts': total_concepts,
            'present_concepts': present_count,
            'coverage_percent': coverage_percent,
            'missing_concepts': missing_concepts[:5],  # Apenas os primeiros 5
            'similarity_score': self.calculate_similarity(original_answer, generated_text) * 100
        }

    def compare_character_count(self) -> Dict:
        """Compara a quantidade de caracteres entre original e gerado"""
        # Contar caracteres do conteúdo original
        original_chars = sum(len(item['answer']) for item in self.original_content.values())
        generated_chars = len(self.generated_content)

        # Calcular diferença percentual
        if original_chars > 0:
            expansion_ratio = generated_chars / original_chars
            difference_percent = ((generated_chars - original_chars) / original_chars) * 100
        else:
            expansion_ratio = 0
            difference_percent = 0

        return {
            'original_chars': original_chars,
            'generated_chars': generated_chars,
            'expansion_ratio': expansion_ratio,
            'difference_percent': difference_percent,
            'status': self._get_expansion_status(expansion_ratio)
        }

    def _get_expansion_status(self, ratio: float) -> str:
        """Determina o status baseado na razão de expansão"""
        if ratio < 0.5:
            return "❌ MUITO COMPRIMIDO"
        elif ratio < 0.8:
            return "⚠️ COMPRIMIDO"
        elif ratio < 1.2:
            return "✅ SIMILAR"
        elif ratio < 2.0:
            return "✅ EXPANDIDO"
        elif ratio < 3.0:
            return "⚠️ MUITO EXPANDIDO"
        else:
            return "❌ EXTREMAMENTE EXPANDIDO"

    def check_content_coverage(self) -> Dict:
        """Verifica cobertura do conteúdo com lógica melhorada"""
        coverage_report = {
            'total_original_items': len(self.original_content),
            'covered_items': 0,
            'missing_items': [],
            'coverage_percentage': 0,
            'details': [],
            'character_analysis': self.compare_character_count()
        }

        for key, item in self.original_content.items():
            presence_analysis = self.check_content_presence(item['answer'], self.generated_content)

            item_report = {
                'key': key,
                'answer': item['answer'],
                'coverage_percent': presence_analysis['coverage_percent'],
                'similarity_score': presence_analysis['similarity_score'],
                'present_concepts': presence_analysis['present_concepts'],
                'total_concepts': presence_analysis['total_concepts'],
                'missing_concepts': presence_analysis['missing_concepts']
            }

            coverage_report['details'].append(item_report)

            # Considera coberto se tem boa cobertura de conceitos OU alta similaridade
            if presence_analysis['coverage_percent'] > 40 or presence_analysis['similarity_score'] > 30:
                coverage_report['covered_items'] += 1
            else:
                coverage_report['missing_items'].append(item_report)

        coverage_report['coverage_percentage'] = (
            coverage_report['covered_items'] / coverage_report['total_original_items']
        ) * 100

        return coverage_report

    def generate_diff_report(self) -> str:
        """Gera relatório melhorado no estilo diff do GitHub"""
        coverage = self.check_content_coverage()
        char_analysis = coverage['character_analysis']

        report = []
        report.append("# 📊 RELATÓRIO DE VALIDAÇÃO DE CONTEÚDO - VERSÃO MELHORADA")
        report.append("=" * 70)
        report.append("")

        # Resumo geral
        report.append("## 📈 RESUMO GERAL")
        report.append(f"- **Total de itens originais:** {coverage['total_original_items']}")
        report.append(f"- **Itens cobertos:** {coverage['covered_items']}")
        report.append(f"- **Cobertura geral:** {coverage['coverage_percentage']:.1f}%")
        report.append("")

        # Análise de caracteres
        report.append("## 📏 ANÁLISE DE CARACTERES")
        report.append(f"- **Caracteres originais:** {char_analysis['original_chars']:,}")
        report.append(f"- **Caracteres gerados:** {char_analysis['generated_chars']:,}")
        report.append(f"- **Razão de expansão:** {char_analysis['expansion_ratio']:.2f}x")
        report.append(f"- **Diferença percentual:** {char_analysis['difference_percent']:+.1f}%")
        report.append(f"- **Status:** {char_analysis['status']}")
        report.append("")

        # Status da cobertura
        if coverage['coverage_percentage'] >= 80:
            status = "✅ EXCELENTE"
        elif coverage['coverage_percentage'] >= 60:
            status = "✅ BOM"
        elif coverage['coverage_percentage'] >= 40:
            status = "⚠️ REGULAR"
        else:
            status = "❌ INSUFICIENTE"

        report.append(f"**Status Geral:** {status}")
        report.append("")

        # Itens bem cobertos
        well_covered = [item for item in coverage['details'] if item['coverage_percent'] >= 60 or item['similarity_score'] >= 40]
        if well_covered:
            report.append("## ✅ CONTEÚDO BEM COBERTO")
            report.append("")
            for item in well_covered:
                report.append(f"### + {item['key']}")
                report.append(f"**Cobertura de conceitos:** {item['coverage_percent']:.1f}% ({item['present_concepts']}/{item['total_concepts']})")
                report.append(f"**Similaridade:** {item['similarity_score']:.1f}%")
                report.append("```diff")
                report.append(f"+ ✅ Conteúdo bem integrado no capítulo")
                report.append("```")
                report.append("")

        # Itens com cobertura parcial
        partial_covered = [item for item in coverage['details']
                          if (20 <= item['coverage_percent'] < 60) and item['similarity_score'] < 40]
        if partial_covered:
            report.append("## ⚠️ CONTEÚDO PARCIALMENTE COBERTO")
            report.append("")
            for item in partial_covered:
                report.append(f"### ~ {item['key']}")
                report.append(f"**Cobertura de conceitos:** {item['coverage_percent']:.1f}% ({item['present_concepts']}/{item['total_concepts']})")
                report.append(f"**Similaridade:** {item['similarity_score']:.1f}%")
                report.append("```diff")
                report.append(f"~ ⚠️ Parte do conteúdo pode estar ausente ou muito reformulado")
                if item['missing_concepts']:
                    report.append("- Conceitos possivelmente ausentes:")
                    for concept in item['missing_concepts']:
                        report.append(f"- {concept}")
                report.append("```")
                report.append("")

        # Itens com baixa cobertura
        low_covered = [item for item in coverage['details']
                      if item['coverage_percent'] < 20 and item['similarity_score'] < 30]
        if low_covered:
            report.append("## ❌ CONTEÚDO COM BAIXA COBERTURA")
            report.append("")
            for item in low_covered:
                report.append(f"### - {item['key']}")
                report.append(f"**Cobertura de conceitos:** {item['coverage_percent']:.1f}% ({item['present_concepts']}/{item['total_concepts']})")
                report.append(f"**Similaridade:** {item['similarity_score']:.1f}%")
                report.append("```diff")
                report.append(f"- ❌ Conteúdo significativamente ausente")
                report.append("- Resposta original:")
                answer_preview = item['answer'][:300] + "..." if len(item['answer']) > 300 else item['answer']
                report.append(f"- {answer_preview}")
                report.append("```")
                report.append("")

        # Recomendações
        report.append("## 🔍 RECOMENDAÇÕES")
        report.append("")

        if coverage['coverage_percentage'] < 60:
            report.append("```diff")
            report.append("- ⚠️ ATENÇÃO: Cobertura abaixo do ideal")
            report.append("+ Revisar itens com baixa cobertura")
            report.append("+ Verificar se informações importantes foram omitidas")
            report.append("+ Considerar incluir mais detalhes dos itens ausentes")
            report.append("```")
        else:
            report.append("```diff")
            report.append("+ ✅ Cobertura satisfatória")
            report.append("+ Conteúdo bem integrado e reformulado")
            report.append("+ Capítulo pronto para revisão final")
            report.append("```")

        # Análise da expansão
        report.append("")
        report.append("## 📊 ANÁLISE DE EXPANSÃO")
        report.append("")

        if char_analysis['expansion_ratio'] < 0.8:
            report.append("```diff")
            report.append("- ⚠️ Conteúdo pode estar muito comprimido")
            report.append("+ Considerar adicionar mais detalhes e contexto")
            report.append("```")
        elif char_analysis['expansion_ratio'] > 2.5:
            report.append("```diff")
            report.append("- ⚠️ Conteúdo pode estar muito expandido")
            report.append("+ Verificar se há informações desnecessárias")
            report.append("```")
        else:
            report.append("```diff")
            report.append("+ ✅ Expansão adequada para um capítulo narrativo")
            report.append("```")

        report.append("")
        report.append("---")
        report.append("*Relatório gerado automaticamente pelo ContentValidator v2.0*")
        report.append("*Lógica melhorada para análise de conteúdo reformulado*")

        return "\n".join(report)

    def run_validation(self, csv_path: str, generated_text: str) -> str:
        """Executa validação completa"""
        print("🔄 Carregando dados originais...")
        self.load_original_data(csv_path)

        print("🔄 Carregando conteúdo gerado...")
        self.load_generated_content(generated_text)

        print("🔄 Analisando cobertura com lógica melhorada...")

        print("🔄 Gerando relatório...")
        report = self.generate_diff_report()

        return report

# 3. PREPARAR DADOS DE EXEMPLO
csv_data = "/caminho_para_csv_com_answers"

# 4. DEFINIR CONTEÚDO GERADO (cole seu capítulo aqui)
generated_text = """
*CONTEÚDO GERADO*
"""

# 5. EXECUTAR VALIDAÇÃO
validator = ContentValidator()

# Executar validação
print("🚀 Iniciando validação com lógica melhorada...")
report = validator.run_validation(csv_data, generated_text)

# 6. EXIBIR RESULTADO
display(Markdown(report))

# 7. FUNÇÃO PARA ANÁLISE DETALHADA
def detailed_analysis(validator):
    """Análise detalhada item por item"""
    print("\n" + "="*60)
    print("ANÁLISE DETALHADA ITEM POR ITEM")
    print("="*60)

    coverage = validator.check_content_coverage()

    for i, item in enumerate(coverage['details'], 1):
        print(f"\n{i}. {item['key']}")
        print(f"   Cobertura de conceitos: {item['coverage_percent']:.1f}%")
        print(f"   Similaridade geral: {item['similarity_score']:.1f}%")
        print(f"   Conceitos presentes: {item['present_concepts']}/{item['total_concepts']}")

        if item['coverage_percent'] >= 60 or item['similarity_score'] >= 40:
            print("   Status: ✅ BEM COBERTO")
        elif item['coverage_percent'] >= 20 or item['similarity_score'] >= 20:
            print("   Status: ⚠️ PARCIALMENTE COBERTO")
        else:
            print("   Status: ❌ BAIXA COBERTURA")

        if item['missing_concepts']:
            print("   Conceitos ausentes:")
            for concept in item['missing_concepts']:
                print(f"   - {concept}")

# Executar análise detalhada
detailed_analysis(validator)

# 8. FUNÇÃO PARA RELATÓRIO DE CONCEITOS
def concept_analysis_report(validator):
    """Relatório focado em conceitos-chave"""
    print("\n" + "="*60)
    print("RELATÓRIO DE CONCEITOS-CHAVE")
    print("="*60)

    for key, item in validator.original_content.items():
        print(f"\n📝 {key}:")
        print(f"Texto original: {item['answer'][:200]}...")

        concepts = validator.extract_key_concepts(item['answer'])
        print(f"Conceitos extraídos: {concepts[:10]}")  # Primeiros 10

        presence = validator.check_content_presence(item['answer'], validator.generated_content)
        print(f"Cobertura: {presence['coverage_percent']:.1f}%")
        print(f"Similaridade: {presence['similarity_score']:.1f}%")

# Executar análise de conceitos
concept_analysis_report(validator)